In [1]:
# Ora che abbiamo finito questa fase, inizia quella "più interessante"
# Inizieremo a prendere dei commenti (raw text) e trasformarli in "vettori", che catturino il significato del testo, 
# o delle richieste, e ci permetta di capire quanto simili o differenti siano tra di loro i pezzi di testo
# Andremo poi a inserire in un database i vettori per facilitare la ricerca di similarità e la costruzione di query in modo efficiente

# Iniziamo con un po' di teoria, studiando perchè per i LLM è necessario creare vettori, per poter creare il nostro vector dataset

In [2]:
# Impariamo come trasformare testo in un forma che ci permetta di comparare quanto siamo simili tra loro matematicamente questi elementi
# Questa funzionalità ha molte applicazioni: noi la useremo per trovare un libro simile alla query fornita
# Per fare questo è necessario comprendere inizialmente un costrutto iniziale chiamato word embedding
# Per comprenderlo consideriamo 7 parole
# queen - king
# girl - boy
# woman - man
# tree

# Pensiamo ora di posizionarli come punti in uno spazio tridimensionale e posizionarli in base alla loro similarità
# Posizioneremo ovviamente queen e king vicini, come girl e boy, come woman e man
# In un angolo posizioneremo tree

# Usando una mappa che mostri le tre coordinate spaziali dei punti,
# - per la dimensione 1, boy e girl sono simili, ma per le altre un po' diverse, forse c'è qualcosa relativo all'età
# - per la dimensione 2, king e queen sono simili e differenti dal resto, e forse questa dimensiona indica qualcosa che 
# ha a che fare con "class" o "nobility"
# - per la dimensione 3, tutti i valori sono molto simili, tranne per tree, quindi potrebbe avere a che fare con "personhood" 

# (nell'esempio: dim 1, 2, 3
# man: 0.61, 0.45, 0.64
# woman: 0.66, 0.49, 0.74
# boy: -0.86, 0.51, 0.65
# girl: -0.83, 0.55, 0.63
# king: 0.42, -0.92, 0.72
# queen: 0.47, -0.84, 0.65
# tree: 0.65, 0.003, -0.92
# )

# I valori assegnati per ogni parole su ogni dimensione sono definiti come word embedding, e rappresentano il significato
# di una parola raggruppando parole simili e creando distanze tra parole che non sono simili 

In [3]:
# Non andremo a creare il nostro word embedding manualmente, che sarebbe un incubo soprattutto per vocaboli molto grandi
# Utilizzeremo modelli di word embedding per fare questo compito, ovvero che costruiscano queste relazioni vedendo
# come le parole si leghino tra loro in un contesto
# Per dimostrare questo vediamo come un modello specifico funziona
# Consideriamo il word2vec, il particolare l'architettura skipgram

# Il modo in cui questo modello impara il word embedding è prendendo tutti gli utilizzi (? = usages) di ogni parola nel 
# set di training e cerca di predirre quali parole siano più adatte a starle vicino in quel contesto (likely to surround it)
# Ad esempio: chiediamo al modello di predirre cosa possa immediatamente precedente a seguire la parola "best" 
# Ovviamente le prime predizioni saranno inaccurate, ma procedendo a vedere sempre più frasi contenenti la parola interessata
# la predizione inizierà ad aggiustarsi sempre più autonomamente e questi tentativi di predizioni diventeranno sempre più 
# vicini all'esatto 

# Consideriamo come prima frase: "The best things you can buy for your dorm room"
# |best| --> |..| --> (w - 1) |The|
#                 --> (w + 1) |things|
# Secondo esempio: "Ryan Gosling tells us the best things about living in LA"
# |best| --> |..| --> (w - 1) |the|
#                 --> (w + 1) |things|
# Terzo esempio: "The best Halloween costume ideas this year"
# |best| --> |..| --> (w - 1) |The|
#                 --> (w + 1) |Halloween|

# Il modello predirrà con maggiore probabilità che la parola che precede "best" è "The" (articolo)
# Con minore sicurezza predirrà come parola successiva "things"
# I pesi che sono stati utilizzati per fare questa previsione saranno il word embedding per "best"
# Questa operazione viene fatta per ogni parola nel vocabolario di allenamento
# Durante l'allenamento il modello apprende, per parole che sono utilizzate in contesti similari,
# a dargli un peso simile all'interno del loro embedding

# Nella pratica, i modelli di word embedding non hanno solo 3 o 5 dimensioni, ma anche centinaia o migliai di dimensioni
# In più, non siamo interessati a conoscere in principio cosa rappresentino le dimensioni, siamo solo interessati
# al risultato finale di avere parole che non sono simili separate e gruppi di parole che sono simili insieme

In [4]:
# I modelli di word embedding sono stati ideati da un po' di tempo e una limitazione in questi è il fatto che 
# facciano collassare tutti i significati di una parola all'interno di uno stesso token, senza considerare il fatto
# che alcune parole possano significare cose diverse in contesti differenti
# Ad esempio: la parola "bank"

# "He checked his bank account" - in senso di istituzione finanziaria
# "The river bank was peaceful" - in senso di una caratteristica naturalistica

# La novità dei modelli recenti sta nel comprendere il significato delle parole inserite in un contesto
# Questa novità è espressa in modelli chiamati transformers e tutti i Cutting Edge LLMs come Chat GPT, Gemini, ...
# sono tutte variazioni di questo tipo di modello 
# Vediamo un veloce esempio per comprendere il funzionamento di questo strumento

# He: (0.37 0.99 0.01 0.08)^T + w(1)
# checked: (0.55 -0.83 0.9 0.71)^T + w(2)
# his: (0.11 0.32 0.47 -0.29)^T + w(3)
# bank: (-0.49 0.5 -0.63 0.41)^T + w(4) 
# account: (-0.17 0.31 0.99 -0.21)^T + w(5)

# Questo è word embedding, come era stato fatto nell'esempio precedente
# Vengono però aggiunti vettori posizionali (positional vectors), w(4) ad esempio, ad ogni embedding
# per indicare la posizione di quelle parole all'interno della frase
# Successivamente, questi due elementi (vettore di word embedding e di posizione) vengono dati in pasto ad
# un meccanismo chiamato di self attention, che utilizza le informazioni dei pesi del word embedding per comprendere
# quanta attention è legata alle altre parole per comprendere il significato della parola

# He: (0.37 0.99 0.01 0.08)^T + w(1) --> (0.54 0.13 0.27 0.04 0.02)^T
# checked: (0.55 -0.83 0.9 0.71)^T + w(2) --> (0.21 0.62 0.03 0.007 0.007)^T
# his: (0.11 0.32 0.47 -0.29)^T + w(3) --> (0.24 0.08 0.58 0.06 0.04)^T
# bank: (-0.49 0.5 -0.63 0.41)^T + w(4) --> (0.02 0.28 0.04 0.41 0.25)^T
# account: (-0.17 0.31 0.99 -0.21)^T + w(5) --> (0.01 0.23 0.02 0.29 0.45)^T

# Vettori di attention: ogni parola ha il suo vettore e ci concentriamo su quello per la parola "bank"
# Ogni valore nel vettore corrisponde ad una parola nella frase, quindi il primo elemento corrisponde a "He", ...
# Le parole che hanno un peso maggiore sono quelle corrispondenti a "checked" (0.28), "bank" (0.41) e "account" (0.25)
# Significa che le parole "checked" e "account" permettono di dare significato (finanziario) alla parola "bank" in questo
# contesto, permettendo al modello di comprendere meglio le similarità
# Altra cosa che viene applicata è una normalizzazione che permette di rendere il lavoro con queste attenzioni più semplice
# Il processo di creazione di vettori di self attention e di normalizazzione è chiamato blocco di encoding del transformer
# Il meccanismo di attenzione permette ai modelli Transformer di fare tante cose interessanti con dati che sono in forma testuale

# L'architettura originale dei Transformer è stata ideata proprio per permette la traduzione di testo da una lingua all'altra
# L'architettura del Transformer prevede due blocchi principali: un encoder e un decoder, e ognuno dei blocchi utilizza un 
# meccanismo di attenzione per gestire testo che viene dato in input
# Nonostante questo, gli obbietivi dei due blocchi sono differenti:
# - Encoder: cerca di carpire più imformazioni possibili su come le parole in una frase nel "linguaggio di origine" siano legate fra loro
# Queste informazioni sono poi inviate al decoder
# - Decoder: ha un compito più complesso, che consiste nel comprendere le relazioni tra parole nel "linguaggio target"
# Utilizza queste informazioni per predirre parola per parola la sequenza più appropriata o likely nel linguaggio target

In [5]:
# L'architettura dei Transformer dalla sua uscita è stata così di successo da essere applicata a modelli sempre più grandi,
# definiti come Large Language Model, o LLM. E' importante conoscere come utilizzare i Transformer ad ogni scopo, se usare
# solo blocchi di encoding, o anche decoding, e altre considerazioni

# I modelli LLM sono tipicamente troppo costosi da allenare da zero (from scratch)
# Una cosa positiva relativa agli LLM sta nel modo in cui imparando in fase di apprendimento, ovvero che sono 
# Natural Language Generalist, hanno una conoscenza profonda di come funziona il linguaggio fornito in training e 
# queste conoscenze possono essere applicate in così tante applicazioni e adattato ad ogni task
# Questo ha portato alla proliferazione di quei modelli pre-allenati
# Molti sono dati open-source sulla piattaforma HuggingFace o in OpenAI

In [6]:
# Vediamo ora alcuni modelli encoder
# Servono per comprendere il significato di un'intera sequenza, come comprendere come le parole si
# legano l'uno all'altra nella frase. Sono addestrati con l'obbiettivo di comprendere questo
# Usiamo ora un modello chiamato RoBERTa che è allenato per comprendere una maschera in un frase, che è 
# una parola mancante
# Quello che RoBERTa fa è tokenizzare questa frase, aggiungendo dei token speciali all'inizio (CLS) e alla fine (SEP) della frase
# I token sono poi passati al modello che ottiene i word embedding per ogni parola e pesata per la sua posizione nella frase

# Esempio di frase di training:
# [CLS] i grew up in paris so i speak [MASK] and english [SEP]
# E-[CLS] E-i E-grew E-up E-in E-paris E-so E-i E-speak E-[MASK] E-and E-english E-[SEP]

# Questo poi è passato ad una serie di blocchi di encoding per ottenere tutti i vettori di self attention
# La parte finale del modello utilizza tutto quello che è stato appreso riguardo la rappresentazione di parole
# nella frase per fare la predizione della parola mancante (mascherata)
# Le predizioni per le prime saranno poco accurate, ma come accaduto per il modello precedente,
# le predizioni miglioreranno con l'aumento degli esempi forniti al modello

#  0.01    0.01    0.001 0.002 0.98     0.003   0.004 0.002   0.003
#     0          0     0    0      1        0      0      0         0
# frame fraternity fraud free french frequent friday friend frustrate  

# Il modello definisce una rappresentazione interna di come il linguaggio funziona e diventa molto accurato
# RoBERTa originariamente fu allenato su 160 GB di testo, è in grado di fare accurate predizioni sul significato
# delle parole in diversi contesti presentati
# Ora che abbiamo lavorato con il modello dei Transformer, di trasformazione di testo in vettori di rappresentazione,
# una cosa interessante che si può fare con i modelli di Coda è passare una sequenza o un documento
# ed estrarre la rappresentazione vettoriale che il modello ha imparato per quella sequenza
# Questo è chiamato document embedding, e come per il word embedding, è un modo per determinare matematicamente
# quanto i documenti siano simili o dissimili tra di loro

In [7]:
# Ritorniamo all'esempio delle descrizioni dei libri
# Partiamo da quattro descrizioni:

# "A heartwarming journey of love and friendship"
# "An ambitious attorney gets entangled in a case which may prove to be more dangerous than anticipated"
# "One of the most meticuluos accounts of the decline and fall of the Roman Empire"
# "A provocative and well-researched take on human relationships"

# Passando queste frasi al modello le convertirebbero nei loro document embedding 

# Vector: (0.54 0.13 0.27 0.04 0.02)^T, ID: 101
# Vector: (0.24 0.08 0.58 0.06 0.04)^T, ID: 102
# Vector: (0.21 0.62 0.03 0.07 0.07)^T, ID: 103
# Vector: (0.02 0.028 0.04 0.41 0.25)^T, ID: 104

# Dato che le descrizioni sono molto dissimili tra di loro, i valori di embedding sono diversi, 
# lo sono anche i valori all'interno dello stesso vettore

# Come facciamo ad identificare un libro specifico?
# Prima cosa che si può fare è inserire i vettori di rappresentazione in un dataset vettoriale
# con una sorta di ID associato ad ogni vettore e altri metadati

# Immaginiamo di voler trovare un libro sull'Impero Romano 
# Viene trasformata la query "A book about the Roman Empire" in un document embedding usando lo stesso
# encoding con cui sono stati definiti i vettori riuniti nel database di riferimento
# Sarà fatta la comparazione con i vettori all'interno del dataset, calcolando un punteggio di
# similarità rispetto ad ogni vettore

# Vector: (0.18 0.67 0.01 0.10 0.16)^T
# Similarity: 0.44 (con ID:101), 0.25 (con ID:102), 0.99 (con ID:103), 0.68 (con ID:104)

# Viene utilizzata la Cosine Similarity per calcolare il punteggio di similarità
# Il vettore di "ID: 103" sembra avere il punteggio massimo di similarità con il vettore relativo alla query
# Non è il compito finale quello di recuperare un vettore, ma dobbiamo fare altro
# Comunque, l'informazione di ID è importante, viene utilizzata come link al vettore nel dataset
# a cui si è fatto riferimento così come agli altri metadati

# "id": "103",
# "title": "History of the Decline and Fall of the Roman Empire - Volume 1",
# "author": "Edward Gibbon",
# "description": "One of the most meticuluos accounts of the decline and fall of the Roman Empire" 

In [8]:
# Potremmo riscontrare un problema: per ora stiamo facendo una ricerca lineare
# La ricerca crescerà con il crescere della dimensione del dataset
# Per risolvere questo ci sono molti algoritmi che possono ridurre lo spazio di ricerca, e lo si fa
# in un modo simile a raggruppare vettori simili all'interno del database, significando che a 
# momento della ricerca, il vettore della query verrà confrontato con insiemi di vettori simili
# Questi raggruppamenti non sono sempre precisi, mettendo in pratica questi algoritmi un trade-off
# tra accuratezza e velocità nella ricerca di similarità tra query e insiemi di vettori 

# Possiamo avere una idea di come queste cose funzionano andando alla pagina di web8 relativa al vector indexing
# E' una compagnia famosa che fornisce database di vettori
# Il vantaggio di usare dataset di vettori è che costruiscono algoritmi all'interno dell'indexing

In [9]:
# Per costruire il nostro Vector search lavoreremo con un framework noto come Langchain
# Langchain è un potente framework disponibile in Python che permette molte applicazioni per i LLM

from langchain_community.document_loaders import TextLoader # Prendera il raw text, il testo delle descrizioni,
# convertendolo in un formato con cui Langchain può lavorare

from langchain_text_splitters import CharacterTextSplitter # Divide le descrizioni in chunck significativi, che nel
# nostro caso saranno rappresentati dalle singole descrizioni dei libri 

#from langchain_openai import OpenAIEmbeddings # Passo successivo è convertire questi chunk in document embedding
# e useremo il metodo OpenAIEmbedding per farlo
# Dato che è a pagamento dopo il periodo di prova, per ora provo l'utiizzo di un modello non a pagamento
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_chroma import Chroma # Dovremo infine inserire tutto in un database vettoriale e useremo Chroma

In [10]:
# Document loader: serve a caricare documenti e renderti utilizzabili per i nostri modelli LLM
# Document Loader -> Text Splitter -> Storage -> Retrieval -> LLM

In [11]:
# Dobbiamo settare un ambiente adatto all'utilizzo di queste applicazioni di AI
# Chiamiamo il modello utilizzando chiamate API, e questo ci porta a dover definire delle API key
# per connetterle al nostro account di openAI per poter utilizzare il modello

# Utilizziamo un primo metodo base per creare le chiavi, usando un paggetto chiamato dot-env
# Si deve creare un file .env in cui inserire le chiavi 
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
# Controlla se la chiave API è presente
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found")

In [13]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [14]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883: A NOVEL THAT READERS and critic...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897: Lewis' work on the nature of lo...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934: ""In The Problem of Pain, C.S. L..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5084,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222: On A Train Journey Home To Nort...
5085,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014: This book tells the tale of a m...
5086,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623: Wisdom to Create a Life of Pass...
5087,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...


In [15]:
# Con tutto l'ambiente pronto, possiamo iniziare a creare il nostro Vector search
# A questo punto, capiamo meglio a cosa era servito creare la feature "tagged_description"
books["tagged_description"]

0       9780002005883: A NOVEL THAT READERS and critic...
1       9780002261982: A new 'Christie for Christmas' ...
2       9780006178736: A memorable, mesmerizing heroin...
3       9780006280897: Lewis' work on the nature of lo...
4       9780006280934: "In The Problem of Pain, C.S. L...
                              ...                        
5084    9788172235222: On A Train Journey Home To Nort...
5085    9788173031014: This book tells the tale of a m...
5086    9788179921623: Wisdom to Create a Life of Pass...
5087    9788185300535: This collection of the timeless...
5088    9789027712059: Since the three volume edition ...
Name: tagged_description, Length: 5089, dtype: object

In [16]:
# Quando creiamo il Vector search e facciamo richieste (query) al database, quello
# che otteniamo in output sono le descrizioni di libri, perchè è ciò che sta dentro 
# al vector database, ma non è quello che vorremmo dare agli utenti del book-recommender
# Ciò che vogliamo restituire agli utenti saranno i titoli e gli autori dei libri
# Quello che possiamo fare è quindi filtrare il dataframe relativo ai libri 
# risultati dalla compatibilità della query con il vector database
# Non è un buon metodo fare string match, è abbastanza lento e può essere impreciso
# Per rendere più "pulito" il passaggio da ottenere descrizione come risultato del match tra query
# e vector dataset, si utilizza la variabile "tagged_description" che oltre alla descrizione contiene
# anche l'identificativo univoco del libro tramite codice isbn. Quindi, si splitta il contenuto della 
# variabile in descrizione e isbn e da quel isbn si fanno a ripescare le informazioni utili all'utente finale
# Si usa quindi l'isbn come filtro per le informazioni utili

In [17]:
# Il textLoader method di Langchain non funziona con i dataframe di pandas
# Quello che facciamo è quindi iniziare col salvare il contenuto di "tagged_description"
# dentro un file testo, solo questa feature

books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)
# Viene salvato come file testo con separazione di nuove linee, senza index e header, deve contenere solo testo
# Ora abbiamo linee separate, con solo testo, che si presentano come "isbn: description"

In [18]:
# Ora separiamo le frasi tra loro
# Facciamo il load delle frasi usando il metodo di Langchain
# Dobbiamo anche instanziare lo splitter di testo

raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
# chunk_lap: non vogliamo che i chunk abbiamo overlap, perchè sono titoli separati
# e non dovrebbero sovrapporsi (overlap)
# chunk_size: lo mettiamo su 0 perchè prima cerca il più vicino separatore all'index number indicato
# dai chunk e se è maggiore di 1 c'è una possibilità che possa non essere splitatto in una nuova linea
# ma che venga splittato su chunk_size. Se invece poniamo chunk_size a 0 siamo sicuri che sarà priorizzata
# la separazione su quello indicato da "separator" e non che splitti sulla chunk_size

# Una volta fatto questo, possiamo utilizzare lo splitter sui documenti caricati
documents = text_splitter.split_documents(raw_documents)
# Questa operazione prenderà del tempo, darà warning sulla lunghezza del chunk
# ma è una cosa normale, di cui abbiamo discusso sopra, quindi non bisogna preoccuparsene

Created a chunk of size 1169, which is longer than the specified 0
Created a chunk of size 1215, which is longer than the specified 0
Created a chunk of size 374, which is longer than the specified 0
Created a chunk of size 310, which is longer than the specified 0
Created a chunk of size 484, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 961, which is longer than the specified 0
Created a chunk of size 189, which is longer than the specified 0
Created a chunk of size 844, which is longer than the specified 0
Created a chunk of size 297, which is longer than the specified 0
Created a chunk of size 198, which is longer than the specified 0
Created a chunk of size 882, which is longer than the specified 0
Created a chunk of size 1089, which is longer than the specified 0
Created a chunk of size 1190, which is longer than the specified 0
Created a chunk of size 305, which is longer than the specified 0
Create

In [19]:
# Verifichiamo che l'operazione sia andata a buon fine caricando il primo documento
documents[0]

# Ha funzionato correttamente, ritorna la prima descrizione

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [20]:
# Inizializza il modello di embeddings
#embeddings = OpenAIEmbeddings(api_key="OPENAI_API_KEY")
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\mazze\AppData\Local\Temp\ipykernel_11864\1954680622.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [107]:
# Dobbiamo creare i document embedding e salvarli in un vector database
db_books = Chroma.from_documents(documents, embedding=embeddings)

In [108]:
query = "A book to teach children about the nature"
docs = db_books.similarity_search(query, k=10) # k: numero massimo di documenti che devono essere restituiti
print(docs)

[Document(id='393281b2-3417-475a-be28-b257bc6e65d3', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'), Document(id='28cbf23c-4ca1-4f07-90c1-670cb0fa26a0', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069: Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'), Document(id='05f26acb-8625-4752-a8b9-c77ffbf321a5', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069: Children will discover the exciting world of their own backyard in this introducti

In [109]:
# Risolviamo il fatto di volere i consigli di letture tramite titoli e libri, e non ritornand le trame dei libri
#books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())] # Errore per isbn seguito da ":"

# Nel caso in cui si rimetta in "tagged_description": "isbn13: description"
isbn_str = docs[0].page_content.split()[0].strip() # Ottieni la prima parola
isbn_str = isbn_str.replace(":", "") # Rimuovi eventuali ":" alla fine
isbn_int = int(isbn_str) # Ora è sicuro convertire in intero
books[books["isbn13"] == isbn_int] # Confronto con l'isbn del dataframe 


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069: Children will discover the exci...


In [124]:
# Ecco ottenuto il primo esempio, il primo libro raccomandato per una unica query
# Per rendere questo processo più sicuro, uniamo tutto in una funzione, in modo da usarla
# per ogni query posta e che ritorni tutte le raccomandazioni richieste
import re

def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame: 
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for rec in recs:
        #books_list += [int(recs[i].page_content.split('"')[0].strip())] # Stessi problemi di prima relativi a ":" finale

        #isbn_str = recs[i].page_content.split('"')[0].strip() # Ottieni la prima parola del contenuto del documento
        #isbn_str = isbn_str.replace(":", "").strip() # Rimuovi eventuali ":" o altri caratteri extra alla fine        
        #isbn_int = int(isbn_str) # Converte la stringa isbn in intero
        #books_list.append(isbn_int) # Aggiunto la stringa isbn alla lista

        match = re.search(r"\b\d{10,13}\b", rec.page_content)
        if match:
            try:
                isbn_int = int(match.group())
                books_list.append(isbn_int)
            except ValueError:
                continue

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [125]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036: Barbara Kingsolver's fifth nove...
1642,9780374522599,0374522596,The Control of Nature,John McPhee,Nature,http://books.google.com/books/content?id=p1qKQ...,The Control of Nature is John McPhee's bestsel...,1990.0,4.24,288.0,3365.0,The Control of Nature,9780374522599: The Control of Nature is John M...
2261,9780446518628,044651862X,The Celestine Prophecy,James Redfield,Fiction,http://books.google.com/books/content?id=UXolx...,You have never read a book like this before --...,1994.0,3.63,247.0,1280.0,The Celestine Prophecy: An Adventure,9780446518628: You have never read a book like...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069: Children will discover the exci...
3748,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,"9780786808373: Introducing your baby to birds,..."
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380: Introduce your babies to birds,..."
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397: Introduce your baby to birds, c..."
3765,9780786819119,0786819111,"Baby Einstein: Water, Water Everywhere","Disney Book Group,",Juvenile Fiction,http://books.google.com/books/content?id=tuAdA...,Charming illustrations and playful rhythmic ve...,2003.0,3.70,10.0,77.0,"Baby Einstein: Water, Water Everywhere",9780786819119: Charming illustrations and play...
3834,9780802431486,0802431488,The 10 Commandments of Parenting,H. Edwin Young,Religion,http://books.google.com/books/content?id=Q1Hxj...,Drawn from years of counseling and the author'...,2005.0,4.00,224.0,25.0,The 10 Commandments of Parenting: The Do's and...,9780802431486: Drawn from years of counseling ...
4331,9781400033850,1400033853,Providence of a Sparrow,Chris Chester,Biography & Autobiography,http://books.google.com/books/content?id=Z7RDD...,Describes the influence of a foundling baby sp...,2004.0,4.32,304.0,218.0,Providence of a Sparrow: Lessons from a Life G...,9781400033850: Describes the influence of a fo...
